# 相机标定

参考：

[【机器视觉】张氏法相机标定](https://zhuanlan.zhihu.com/p/24651968)

## 获取棋盘格角点世界坐标

我们首先要确定棋盘格的两个属性：
- 棋盘格的内角点个数（行、列）
- 棋盘格每个小格子的边长（单位：毫米、厘米等）

根据这两个属性我们生成坐标数组：

In [ ]:
w, h = self.chessboard_size
# 创建角点的世界坐标 (z=0)
objp = np.zeros((w * h, 3), np.float32)
# np.mgrid[0:w, 0:h]返回一个2xwxh的数组，然后通过.T转置为hxwx2，再reshape(-1,2)变成(w*h, 2)的数组。这样，每一行就是一个角点的(x, y)坐标，其中x从0到w-1，y从0到h-1
objp[:, :2] = np.mgrid[0:w, 0:h].T.reshape(-1, 2)
# objp内容：
# x坐标从0到7，y坐标从0到4，z坐标全为0
#[[0. 0. 0.]
# [1. 0. 0.]
# [2. 0. 0.]
# ···
# [6. 4. 0.]
# [7. 4. 0.]]
# 将网格坐标转换为物理坐标
return objp * self.square_size

## 获取棋盘格角点像素坐标

将事先拍摄的多张棋盘格图片读入，转换为灰度图，然后使用`cv2.findChessboardCorners`函数检测角点位置，并使用`cv2.cornerSubPix`函数对角点位置进行亚像素级精确化

函数原型：

`cv2.findChessboardCorners(image, patternSize, flags)`

参数说明：

- image：输入图像，必须为灰度图
- patternSize：棋盘格内角点的数量，格式为(width, height)
- flags：检测标志，一般填None

返回值：

- retval：布尔值，表示是否成功检测到角点
- corners：检测到的角点坐标，格式为(N, 1, 2)的数组

函数原型：

`cv2.cornerSubPix(image, corners, winSize, zeroZone, criteria)`

参数说明：

- image：输入图像，必须为灰度图
- corners：初始角点坐标，格式为(N, 1, 2)的数组
- winSize：搜索窗口大小，一般设置为(11, 11)
- zeroZone：零区域大小，一般设置为(-1, -1)，表示没有零区域
- criteria：迭代终止条件，一般设置为(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

返回值：

- corners：精确化后的角点坐标，格式为(N, 1, 2)的数组

In [ ]:
# 准备世界坐标系点
objp = self.prepare_object_points() # 调用上面的函数
# 初始化存储列表
obj_points = []  # 3D点在世界坐标系中
img_points = []  # 2D点在图像平面中
# ···
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 查找棋盘格角点
ret, corners = cv2.findChessboardCorners(gray, self.chessboard_size, None)
if ret: # 找到角点
    # 提高角点检测精度
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    corners_refined = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
    # 存储点
    obj_points.append(objp)
    img_points.append(corners_refined)
    # 显示和保存角点检测结果
    img_with_corners = img.copy()
    cv2.drawChessboardCorners(img_with_corners, self.chessboard_size, corners_refined, ret)
# ···
return obj_points, img_points

## 相机标定

知道了世界坐标和像素坐标后，我们就可以使用`cv2.calibrateCamera`函数进行相机标定，求解相机的内参矩阵和畸变系数

函数原型：

`cv2.calibrateCamera(objectPoints, imagePoints, imageSize, cameraMatrix, distCoeffs)`

参数说明：

- objectPoints：世界坐标点，格式为列表，每个元素为(N, 3)的数组
- imagePoints：像素坐标点，格式为列表，每个元素为(N, 2)的数组
- imageSize：图像尺寸，格式为(width, height)
- cameraMatrix：相机内参矩阵，格式为(3, 3)的数组，如果未知则传入None
- distCoeffs：畸变系数，格式为(1, 5)的数组，如果未知则传入None

返回值：

- retval：标定结果的均方根误差
- cameraMatrix：相机内参矩阵，格式为(3, 3)的数组
- distCoeffs：畸变系数，格式为(1, 5)的数组
- rvecs：旋转向量，格式为列表，每个元素为(3, 1)的数组
- tvecs：平移向量，格式为列表，每个元素为(3, 1)的数组

In [ ]:
# 获取角点
obj_points, img_points = self.find_chessboard_corners(image_dir, image_type, save_dir=save_dir)
# 获取图像尺寸
sample_image = cv2.imread(glob.glob(os.path.join(image_dir, f"*.{image_type}"))[0])
image_size = (sample_image.shape[1], sample_image.shape[0])  # (width, height)
# 相机标定
ret, self.camera_matrix, self.dist_coeffs, self.rvecs, self.tvecs = cv2.calibrateCamera(
    obj_points, img_points, image_size, None, None)

## 计算重投影误差

重投影误差用于评估相机标定的精度，表示通过标定得到的参数重新投影到图像平面上的点与实际检测到的点之间的距离误差。重投影误差越小，说明标定结果越准确

函数原型：

`cv2.projectPoints(objectPoints, rvec, tvec, cameraMatrix, distCoeffs)`

用于将3D点投影到2D图像平面上

参数说明：

- objectPoints：3D点，格式为(N, 3)的数组
- rvec：旋转向量，格式为(3, 1)的数组，将相机标定获得的旋转向量传入
- tvec：平移向量，格式为(3, 1)的数组，将相机标定获得的平移向量传入
- cameraMatrix：相机内参矩阵，格式为(3, 3)的数组
- distCoeffs：畸变系数，格式为(1, 5)的数组

返回值：

- imagePoints：投影到图像平面上的2D点，格式为(N, 1, 2)的数组
- jacobian：雅可比矩阵

函数原型：

`cv2.norm(src1, src2, normType)`

用于计算两个数组之间的范数，一般用于计算误差

参数说明：

- src1：第一个输入数组
- src2：第二个输入数组
- normType：范数类型，一般使用cv2.NORM_L2表示L2范数

返回值：

- normValue：计算得到的范数值

下面是计算重投影误差的代码示例：

In [ ]:
def _compute_reprojection_error(self, obj_points: List, img_points: List) -> float:
        """计算平均重投影误差"""
        total_error = 0
        for i in range(len(obj_points)):
            img_points_repro, _ = cv2.projectPoints(
                obj_points[i], self.rvecs[i], self.tvecs[i], 
                self.camera_matrix, self.dist_coeffs)
            error = cv2.norm(img_points[i], img_points_repro, cv2.NORM_L2) / len(img_points_repro)
            total_error += error
        return total_error / len(obj_points)

## 使用获得的内参矩阵与畸变系数

### 根据当前图像尺寸修正得到新的相机矩阵和ROI

函数原型：

`new_K, roi = cv2.getOptimalNewCameraMatrix(K, D, (w, h), alpha, (w, h))`

参数说明：

- K: 原始相机内参（3×3矩阵）
- D: 畸变系数（k1, k2, p1, p2, k3）
- (w, h): 图像分辨率（宽度，高度）
- alpha: 调整视野（0~1），决定是否保留黑边
- (w, h): 计算新相机矩阵的目标尺寸

alpha参数的作用：alpha 决定去畸变后如何调整视野

- alpha=0 → 最大去畸变，无黑边但可能裁剪图像
- alpha=1 → 保留所有像素，可能带黑边
- alpha=0.5 → 介于两者之间

返回值：

- new_K: 去畸变后的相机内参矩阵（3×3）
- roi: (x, y, width, height)，推荐的裁剪区域

### 去畸变

函数原型：

`dst = cv2.undistort(src, cameraMatrix, distCoeffs, dst, newCameraMatrix)`

参数说明：

- src: 输入畸变图像
- cameraMatrix: 原始相机内参矩阵（3×3）
- distCoeffs: 畸变系数向量 (k1, k2, p1, p2[, k3[, k4, k5, k6]])
- dst: 可选参数，输出去畸变图像，填None
- newCameraMatrix: 可选参数，新的相机内参矩阵

返回值：

- dst: 去畸变后的图像

有时候我们只想得到图像上特征点、角点的矫正坐标

`dst = cv2.undistortPoints(src, cameraMatrix, distCoeffs, dst, R, P)`

参数说明：

- src: 输入点集，形状为 N×1×2 或 1×N×2 的数组
- cameraMatrix: 相机内参矩阵（3×3）
- distCoeffs: 畸变系数向量
- dst: 可选参数，输出去畸变点集
- R: 可选参数，校正变换矩阵（3×3）
- P: 可选参数，新的相机内参矩阵（3×3），使用getOptimalNewCameraMatrix()函数获得，填写该参数最后函数返回值为像素坐标

返回值：

- dst: 去畸变后的点集（归一化坐标）

归一化坐标是去除了相机内参影响的坐标，将像素坐标转换到相机坐标系下的归一化平面（Z=1平面）